In [1]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
import string
import nltk
import sklearn
import re
import mglearn as mglearn

from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import CountVectorizer

/home/swagmaster/anaconda3/envs/atab/lib/python3.7/site-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/home/swagmaster/anaconda3/envs/atab/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


In [2]:
train = pd.read_csv('train2electricboogaloo.csv')

train.Summary.fillna('', inplace=True)
train.Text.fillna('', inplace=True)
train.SumTxt.fillna('', inplace=True)

In [3]:
train_set, test_set = train_test_split(train, test_size = 0.2, random_state = 42, stratify = train['Score'])
train_text, train_score = train_set['SumTxt'], train_set['Score']
test_text, test_score = test_set['SumTxt'], test_set['Score']

In [4]:
text, score = train['SumTxt'], train['Score']

In [5]:
stemmer = SnowballStemmer("english")
wnl = WordNetLemmatizer()
stop_words = set(stopwords.words("english"))

In [ ]:
contractions_dict = { 
    "ain\'t": "am not",
    "aren\'t": "are not",
    "can\'t": "cannot",
    "can\'t\'ve": "cannot have",
    "\'cause": "because",
    "could\'ve": "could have",
    "couldn\'t": "could not",
    "couldn\'t\'ve": "could not have",
    "didn\'t": "did not",
    "doesn\'t": "does not",
    "don\'t": "do not",
    "hadn\'t": "had not",
    "hadn\'t\'ve": "had not have",
    "hasn\'t": "has not",
    "haven\'t": "have not",
    "he\'d": "he had",
    "he\'d\'ve": "he would have",
    "he\'ll": "he shall",
    "he\'ll\'ve": "he shall have",
    "he\'s": "he has",
    "how\'d": "how did",
    "how\'d\'y": "how do you",
    "how\'ll": "how will",
    "how\'s": "how has",
    "I\'d": "I had",
    "I\'d\'ve": "I would have",
    "I\'ll": "I will",
    "I\'ll\'ve": "I will have",
    "I\'m": "I am",
    "I\'ve": "I have",
    "isn\'t": "is not",
    "it\'d": "it would",
    "it\'d\'ve": "it would have",
    "it\'ll": "it will",
    "it\'ll\'ve": "it will have",
    "it\'s": "it is",
    "let\'s": "let us",
    "ma\'am": "madam",
    "mayn\'t": "may not",
    "might\'ve": "might have",
    "mightn\'t": "might not",
    "mightn\'t\'ve": "might not have",
    "must\'ve": "must have",
    "mustn\'t": "must not",
    "mustn\'t\'ve": "must not have",
    "needn\'t": "need not",
    "needn\'t\'ve": "need not have",
    "o\'clock": "of the clock",
    "oughtn\'t": "ought not",
    "oughtn\'t\'ve": "ought not have",
    "shan\'t": "shall not",
    "sha\'n\'t": "shall not",
    "shan\'t\'ve": "shall not have",
    "she\'d": "she had",
    "she\'d\'ve": "she would have",
    "she\'ll": "she will",
    "she\'ll\'ve": "she will have",
    "she\'s": "she is",
    "should\'ve": "should have",
    "shouldn\'t": "should not",
    "shouldn\'t\'ve": "should not have",
    "so\'ve": "so have",
    "so\'s": "so as",
    "that\'d": "that would",
    "that\'d\'ve": "that would have",
    "that\'s": "that is",
    "there\'d": "there had",
    "there\'d\'ve": "there would have",
    "there\'s": "there has",
    "they\'d": "they had",
    "they\'d\'ve": "they would have",
    "they\'ll": "they will",
    "they\'ll\'ve": "they will have",
    "they\'re": "they are",
    "they\'ve": "they have",
    "to\'ve": "to have",
    "wasn\'t": "was not",
    "we\'d": "we had",
    "we\'d\'ve": "we would have",
    "we\'ll": "we will",
    "we\'ll\'ve": "we will have",
    "we\'re": "we are",
    "we\'ve": "we have",
    "weren\'t": "were not",
    "what\'ll": "what will",
    "what\'ll\'ve": "what will have",
    "what\'re": "what are",
    "what\'s": "what is",
    "what\'ve": "what have",
    "when\'s": "when is",
    "when\'ve": "when have",
    "where\'d": "where did",
    "where\'s": "where has",
    "where\'ve": "where have",
    "who\'ll": "who will",
    "who\'ll\'ve": "who will have",
    "who\'s": "who is",
    "who\'ve": "who have",
    "why\'s": "why is",
    "why\'ve": "why have",
    "will\'ve": "will have",
    "won\'t": "will not",
    "won\'t\'ve": "will not have",
    "would\'ve": "would have",
    "wouldn\'t": "would not",
    "wouldn\'t\'ve": "would not have",
    "y\'all": "you all",
    "y\'all\'d": "you all would",
    "y\'all\'d\'ve": "you all would have",
    "y\'all\'re": "you all are",
    "y\'all\'ve": "you all have",
    "you\'d": "you would",
    "you\'d\'ve": "you would have",
    "you\'ll": "you will",
    "you\'ll\'ve": "you will have",
    "you\'re": "you are",
    "you\'ve": "you have"
}

In [ ]:
def decontracted(phrase):
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase

In [ ]:
def expand_contractions(s, contractions_dict=contractions_dict):
    x = [contractions_dict[con] if con in contractions_dict else con for con in s.split()]
    x = ' '.join(x)
    return decontracted(x)

In [ ]:
def text_process_stm_cont(mess): #ss, swr
    result = ''.join([i for i in mess if not i.isdigit()])
#     nopunc = ' '.join([word for word in result.split() if word.lower() not in stop_words])
    nopunc = expand_contractions(nopunc)
    nopunc = ' '.join([word for word in nopunc.split() if word.lower() not in stop_words])
    nopunc = nopunc.translate(str.maketrans('', '', string.punctuation))
    nopunc = [word.lower() for word in nopunc.split()]
    nopunc = [word for word in result.split() if len(word) != 1]
    return [stemmer.stem(word) for word in nopunc]

In [ ]:
def text_process_wnl_cont(mess): #ss, swr
    result = ''.join([i for i in mess if not i.isdigit()])
#     nopunc = ' '.join([word for word in result.split() if word.lower() not in stop_words])
    nopunc = expand_contractions(nopunc)
    nopunc = ' '.join([word for word in nopunc.split() if word.lower() not in stop_words])
    nopunc = nopunc.translate(str.maketrans('', '', string.punctuation))
    nopunc = [word.lower() for word in nopunc.split()]
    nopunc = [word for word in result.split() if len(word) != 1]
    return [wnl.lemmatize(word) for word in nopunc]

In [ ]:
def text_process_stm_cont_swnr(mess): #ss, swr
    result = ''.join([i for i in mess if not i.isdigit()])
#     nopunc = ' '.join([word for word in result.split() if word.lower() not in stop_words])
    nopunc = expand_contractions(nopunc)
#     nopunc = ' '.join([word for word in nopunc.split() if word.lower() not in stop_words])
    nopunc = nopunc.translate(str.maketrans('', '', string.punctuation))
    nopunc = [word.lower() for word in nopunc.split()]
    nopunc = [word for word in result.split() if len(word) != 1]
    return [stemmer.stem(word) for word in nopunc]

In [ ]:
def text_process_wnl_cont_swnr(mess): #ss, swr
    result = ''.join([i for i in mess if not i.isdigit()])
#     nopunc = ' '.join([word for word in result.split() if word.lower() not in stop_words])
    nopunc = expand_contractions(nopunc)
#     nopunc = ' '.join([word for word in nopunc.split() if word.lower() not in stop_words])
    nopunc = nopunc.translate(str.maketrans('', '', string.punctuation))
    nopunc = [word.lower() for word in nopunc.split()]
    nopunc = [word for word in result.split() if len(word) != 1]
    return [wnl.lemmatize(word) for word in nopunc]

In [6]:
def text_process_wnl(mess): #ss, swr
    result = ''.join([i for i in mess if not i.isdigit()])
    nopunc = result.translate(str.maketrans('', '', string.punctuation))
    nopunc2 = [word for word in nopunc.split() if word.lower() not in stop_words]
    nopunc2 = [word.lower() for word in nopunc2]
    nopunc2 = [word for word in nopunc2 if len(word) != 1]
    return [stemmer.stem(word) for word in nopunc2]

def text_process_stm(mess): #ss, swr
    result = ''.join([i for i in mess if not i.isdigit()])
    nopunc = result.translate(str.maketrans('', '', string.punctuation))
    nopunc2 = [word for word in nopunc.split() if word.lower() not in stop_words]
    nopunc2 = [word.lower() for word in nopunc2]
    nopunc2 = [word for word in nopunc2 if len(word) != 1]
    return [wnl.lemmatize(word) for word in nopunc2]

In [7]:
vect = CountVectorizer(analyzer = text_process_stm).fit(train['SumTxt'])
vect2 = CountVectorizer(analyzer = text_process_stm2).fit(train['SumTxt'])

In [8]:
feature_names = vect.get_feature_names()
feature_names2 = vect2.get_feature_names()

In [9]:
len(feature_names)

145737

In [10]:
len(feature_names2)

170799

In [11]:
print("First 2000 features:\n{}".format(feature_names[:200]))

First 2000 features:
['\x06\x0f\x07\x0f\x07\x0f\x07aumllt', '\x10i', 'a', 'aa', 'aaa', 'aaaa', 'aaaaa', 'aaaaaa', 'aaaaaaaaaa', 'aaaaaaaaaaa', 'aaaaaaaaaaaa', 'aaaaaaaaaaaaaa', 'aaaaaaaaaaaaaaa', 'aaaaaaaaaaaaaaaaaaaargh', 'aaaaaaaaaaaaaaaaacccccccckkkkkkkkkk', 'aaaaaaaaaaacccckkkkk', 'aaaaaaaaagghh', 'aaaaaaah', 'aaaaaaahhhhhh', 'aaaaaaarrrrrggghhh', 'aaaaaaggggggghhhhhhhh', 'aaaaaagggghhhh', 'aaaaaagh', 'aaaaaah', 'aaaaaahhh', 'aaaaaahhhh', 'aaaaaahhhhh', 'aaaaaahhhhhyaaaaaa', 'aaaaaawwwwwwwwww', 'aaaaah', 'aaaaahhh', 'aaaaallll', 'aaaaawsom', 'aaaah', 'aaaahhh', 'aaaahhhh', 'aaaahhhhh', 'aaaahhhhhh', 'aaaallll', 'aaaamaz', 'aaaammmmaaaaazziiiinnnnggg', 'aaaand', 'aaaannnndddgolazo', 'aaaarrrrghh', 'aaaarrrrrgggghhhhh', 'aaagh', 'aaah', 'aaahhh', 'aaahhhh', 'aaahhhhhh', 'aaahmaz', 'aaamaz', 'aabout', 'aabsolut', 'aachen', 'aad', 'aadd', 'aadmit', 'aadp', 'aadult', 'aaf', 'aafco', 'aafcoa', 'aah', 'aahey', 'aahh', 'aahhh', 'aain', 'aakaufmanerthlinknet', 'aakg', 'aalmost', 'aaloo', 'a

In [12]:
print("First 2000 features:\n{}".format(feature_names2[:200]))

First 2000 features:
['\x06\x0f\x07\x0f\x07\x0f\x07aumllt', '\x10i', 'aa', 'aaa', 'aaaa', 'aaaaa', 'aaaaaa', 'aaaaaaaaaa', 'aaaaaaaaaaa', 'aaaaaaaaaaaa', 'aaaaaaaaaaaaaa', 'aaaaaaaaaaaaaaa', 'aaaaaaaaaaaaaaaaaaaargh', 'aaaaaaaaaaaaaaaaacccccccckkkkkkkkkk', 'aaaaaaaaaaacccckkkkk', 'aaaaaaaaagghh', 'aaaaaaah', 'aaaaaaahhhhhh', 'aaaaaaarrrrrggghhh', 'aaaaaaggggggghhhhhhhh', 'aaaaaagggghhhh', 'aaaaaagh', 'aaaaaah', 'aaaaaahhh', 'aaaaaahhhh', 'aaaaaahhhhh', 'aaaaaahhhhhyaaaaaa', 'aaaaaawwwwwwwwww', 'aaaaah', 'aaaaahhh', 'aaaaallll', 'aaaaawsome', 'aaaah', 'aaaahhh', 'aaaahhhh', 'aaaahhhhh', 'aaaahhhhhh', 'aaaahs', 'aaaallll', 'aaaamazing', 'aaaammmmaaaaazziiiinnnnggg', 'aaaand', 'aaaannnndddgolazo', 'aaaarrrrghh', 'aaaarrrrrgggghhhhh', 'aaagh', 'aaah', 'aaahhh', 'aaahhhh', 'aaahhhhhh', 'aaahhhs', 'aaahmazing', 'aaahs', 'aaamazing', 'aabout', 'aabsolutely', 'aachen', 'aad', 'aadd', 'aadmit', 'aadp', 'aadults', 'aaf', 'aafco', 'aafcoa', 'aafes', 'aah', 'aahey', 'aahh', 'aahhhs', 'aahing', 'aa